In [2]:
# pip install vaderSentiment

     |████████████████████████████████| 133kB 1.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import glob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [7]:
files = []
for file in glob.glob("../data/Official Tweets/*"):
    files.append(file)

In [22]:
for file in files:
    print("Reading file - " + file)
    dataset = pd.read_csv(file)
    analyzer = SentimentIntensityAnalyzer()
    dataset["sentiment_compound_score"] = dataset["Text"].apply(lambda x : analyzer.polarity_scores(str(x)))
    df = pd.json_normalize(dataset['sentiment_compound_score'])
    dataset_updated = pd.merge(dataset, df, how="inner", right_index = True, left_index= True)
    dataset_updated["sentiment"] = dataset_updated["compound"].apply(lambda x : "Positive" if (x > 0.05) else ("Neutral" if (x > -0.05 and x < 0.05) else "Negative"))
    display(dataset_updated.head(2))
    display(dataset_updated["sentiment"].value_counts())
#     dataset_updated["sentiment"].value_counts().plot(kind="bar")

Reading file - ../data/Official Tweets/24-Oct.csv


,Unnamed: 0,Datetime,Tweet Id,Text,Username,sentiment_compound_score,neg,neu,pos,compound,sentiment
0,0,2021-10-24 21:46:00+00:00,1452390945656168448,❤️\n\n#MUFC | #MUNLIV https://t.co/5DAnEB52lc,ManUtd,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.0,0.0,0.0,Neutral
1,1,2021-10-24 20:31:00+00:00,1452372071346958336,Our captain spoke to #MUTV after #MUNLIV.\n\n#...,ManUtd,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.0,0.0,0.0,Neutral


Neutral     30
Positive    17
Negative     2
Name: sentiment, dtype: int64

Reading file - ../data/Official Tweets/02-Oct.csv


,Unnamed: 0,Datetime,Tweet Id,Text,Username,sentiment_compound_score,neg,neu,pos,compound,sentiment
0,0,2021-10-02 22:28:00+00:00,1444428982238134273,Landmark reached in 58 #PL appearances 🎯\n\n#M...,ManUtd,"{'neg': 0.0, 'neu': 0.787, 'pos': 0.213, 'comp...",0.0,0.787,0.213,0.1779,Positive
1,1,2021-10-02 21:02:00+00:00,1444407339986083844,Up and running in 2021/22 👊\n\n@AnthonyMartial...,ManUtd,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000,Neutral


...

,Unnamed: 0,Datetime,Tweet Id,Text,Username,sentiment_compound_score,neg,neu,pos,compound,sentiment
0,0,2021-10-16 18:26:00+00:00,1449441511129554946,"As always, thank you to our travelling fans fo...",ManUtd,"{'neg': 0.0, 'neu': 0.606, 'pos': 0.394, 'comp...",0.000,0.606,0.394,0.8442,Positive
1,1,2021-10-16 17:51:45+00:00,1449432894708060160,The boss has shared his thoughts after today's...,ManUtd,"{'neg': 0.195, 'neu': 0.649, 'pos': 0.156, 'co...",0.195,0.649,0.156,-0.1531,Negative


Neutral     20
Positive    14
Negative     3
Name: sentiment, dtype: int64

Reading file - ../data/Official Tweets/29-Aug.csv


,Unnamed: 0,Datetime,Tweet Id,Text,Username,sentiment_compound_score,neg,neu,pos,compound,sentiment
0,0,2021-08-29 23:57:00+00:00,1432130192428843009,𝗘𝗱𝗶 𝗶𝘀 𝗯𝗮𝗰𝗸! 🤩\n\n#MUFC | @ECavaniOfficial htt...,ManUtd,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000,Neutral
1,1,2021-08-29 23:02:00+00:00,1432116351510925316,Your support at Molineux on Sunday was incredi...,ManUtd,"{'neg': 0.0, 'neu': 0.857, 'pos': 0.143, 'comp...",0.0,0.857,0.143,0.4574,Positive


Positive    35
Neutral     31
Negative     5
Name: sentiment, dtype: int64